In [2]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
# parameters
learning_rate = 0.5
batch_size = 10

In [5]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100.0%

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [6]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
w1 = torch.nn.Parameter(torch.Tensor(784, 30)).to(device)
b1 = torch.nn.Parameter(torch.Tensor(30)).to(device)
w2 = torch.nn.Parameter(torch.Tensor(30, 10)).to(device)
b2 = torch.nn.Parameter(torch.Tensor(10)).to(device)

In [8]:
torch.nn.init.normal_(w1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(w2)
torch.nn.init.normal_(b2)

Parameter containing:
tensor([ 0.3078, -1.9857,  1.0512,  1.5122, -1.0199, -0.7402, -1.3111,  0.6142,
        -0.6474,  0.1758], requires_grad=True)

In [9]:
def sigmoid(x):
    #  sigmoid function
    return 1.0 / (1.0 + torch.exp(-x))
    # return torch.div(torch.tensor(1), torch.add(torch.tensor(1.0), torch.exp(-x)))

Now define a function to calculate the devariative of sigmoid function.

In [10]:
def sigmoid_prime(x):
    # derivative of the sigmoid function
    return sigmoid(x) * (1 - sigmoid(x))

In [11]:
X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)[:1000]
Y_test = mnist_test.targets.to(device)[:1000]
i = 0
while not i == 10000:
    for X, Y in data_loader:
        i += 1

        # forward
        X = X.view(-1, 28 * 28).to(device)
        Y = torch.zeros((batch_size, 10)).scatter_(1, Y.unsqueeze(1), 1).to(device)    # one-hot
        l1 = torch.add(torch.matmul(X, w1), b1)
        a1 = sigmoid(l1)
        l2 = torch.add(torch.matmul(a1, w2), b2)
        y_pred = sigmoid(l2)
        # loss
        diff = y_pred - Y

        # Back prop: Loss derivative calculation (chain rule)
        d_l2 = diff * sigmoid_prime(l2)
        d_b2 = d_l2
        d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_l2)

        d_a1 = torch.matmul(d_l2, torch.transpose(w2, 0, 1))
        d_l1 = d_a1 * sigmoid_prime(l1)
        d_b1 = d_l1
        d_w1 = torch.matmul(torch.transpose(X, 0, 1), d_l1)

        # update parameters
        w1 = w1 - learning_rate * d_w1
        b1 = b1 - learning_rate * torch.mean(d_b1, 0)
        w2 = w2 - learning_rate * d_w2
        b2 = b2 - learning_rate * torch.mean(d_b2, 0)

        if i % 1000 == 0:
            l1 = torch.add(torch.matmul(X_test, w1), b1)
            a1 = sigmoid(l1)
            l2 = torch.add(torch.matmul(a1, w2), b2)
            y_pred = sigmoid(l2)
            acct_mat = torch.argmax(y_pred, 1) == Y_test
            acct_res = acct_mat.sum()
            print("accuracy: ",acct_res.item(),"/1000")

        if i == 10000:
            break

/usr/local/lib/python3.12/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.12/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


accuracy:  793 /1000
accuracy:  847 /1000
accuracy:  876 /1000
accuracy:  883 /1000
accuracy:  884 /1000
accuracy:  899 /1000
accuracy:  883 /1000
accuracy:  914 /1000
accuracy:  892 /1000
accuracy:  900 /1000


Explaination of the loss computing process:
1. **Gradient of the Loss with Respect to `l2`**:
   - The gradient of the loss with respect to `l2` is computed by multiplying the difference (`diff`) by the derivative of the sigmoid function applied to `l2`. This step uses the chain rule to propagate the gradient through the sigmoid activation function.

2. **Gradient of the Loss with Respect to `b2`**:
   - The gradient of the loss with respect to `b2` is the same as the gradient of the loss with respect to `l2` because the bias term is added directly to the linear transformation.

3. **Gradient of the Loss with Respect to `w2`**:
   - The gradient of the loss with respect to `w2` is computed by multiplying the transpose of `a1` by the gradient of the loss with respect to `l2`. This step uses the chain rule to propagate the gradient through the linear transformation.

4. **Gradient of the Loss with Respect to `a1`**:
   - The gradient of the loss with respect to `a1` is computed by multiplying the gradient of the loss with respect to `l2` by the transpose of `w2`. This step uses the chain rule to propagate the gradient backward through the weights of the second layer.

5. **Gradient of the Loss with Respect to `l1`**:
   - The gradient of the loss with respect to `l1` is computed by multiplying the gradient of the loss with respect to `a1` by the derivative of the sigmoid function applied to `l1`. This step uses the chain rule to propagate the gradient through the sigmoid activation function.

6. **Gradient of the Loss with Respect to `b1`**:
   - The gradient of the loss with respect to `b1` is the same as the gradient of the loss with respect to `l1` because the bias term is added directly to the linear transformation.

7. **Gradient of the Loss with Respect to `w1`**:
   - The gradient of the loss with respect to `w1` is computed by multiplying the transpose of `X` by the gradient of the loss with respect to `l1`. This step uses the chain rule to propagate the gradient through the linear transformation.

8. **Parameter Updates**:
   - The weights and biases are updated by subtracting the product of the learning rate and the corresponding gradients. This step moves the parameters in the direction that reduces the loss.

By following these steps, the backpropagation algorithm efficiently computes the gradients and updates the model parameters to minimize the loss function, allowing the neural network to learn from the data.